# Problem 1

In [72]:
import pandas as pd
import numpy as np
from numpy import random

class CorruptQueue:
    def __init__ (self, df):
        self.RegularQueue = []
        self.VIPStack = []
        self.supervisor = False
        self.df = df
        self.df.columns=['event', 'client', 'type']
        
    def read(self):
        for idx, row in self.df.iterrows():
            if row.event == "arrive":
                print("Supervisor is present")
                [self.RegularQueue.append(self.VIPStack.pop()) for x in range(len(self.VIPStack))]
                self.supervisor = True
            elif row.event == "leave":
                print("Supervisor not here")
                self.supervisor = False
            elif row.event == "lineup":
                if self.supervisor == True or row.type == "regular":
                    self.RegularQueue.append([row.client, row.type])
                    print(row.type,"client", row.client, "lines up at RegularQueue")
                elif self.supervisor == False and row.type == "VIP":
                    self.VIPStack.append([row.client, row.type])
                    print(row.type, "client", row.client, " lines up at VIPStack")
            elif row.event == "serve":
                if self.VIPStack != []:
                    print("Now serving", self.VIPStack[-1][0],"from VIPStack")
                    self.VIPStack.pop()
                else:
                    print("Now serving", self.RegularQueue[0][0],"from RegularQueue")
                    self.RegularQueue.pop(0)

In [73]:
df = pd.read_csv("officeinput.txt", header=None)
a = CorruptQueue(df)
a.read()

regular client John lines up at RegularQueue
regular client Bob lines up at RegularQueue
regular client Tom lines up at RegularQueue
VIP client Sarah  lines up at VIPStack
VIP client Marie  lines up at VIPStack
VIP client Joan  lines up at VIPStack
Now serving Joan from VIPStack
Now serving Marie from VIPStack
Supervisor is present
Now serving John from RegularQueue
Now serving Bob from RegularQueue
Now serving Tom from RegularQueue
VIP client Bea lines up at RegularQueue
regular client Hank lines up at RegularQueue
Now serving Sarah from RegularQueue
Now serving Bea from RegularQueue
Now serving Hank from RegularQueue
Supervisor not here
regular client Art lines up at RegularQueue
VIP client Daisy  lines up at VIPStack
regular client Marius lines up at RegularQueue
VIP client Dane  lines up at VIPStack
Now serving Dane from VIPStack
Supervisor is present
Now serving Art from RegularQueue
Now serving Marius from RegularQueue
Now serving Daisy from RegularQueue
Supervisor not here


# Problem 2

In [74]:
class CorruptQueue:
    def __init__ (self, df):
        self.RegularQueue = []
        self.VIPStack = []
        self.supervisor = False
        self.df = df
        self.df.columns=['event', 'client', 'type']
        
    def read(self):
        for idx, row in self.df.iterrows():
            if row.event == "arrive":
                print("Supervisor is present", file=open(output, "a"))
                [self.RegularQueue.append(self.VIPStack.pop()) for x in range(len(self.VIPStack))]
                self.supervisor = True
            elif row.event == "leave":
                print("Supervisor not here", file=open(output, "a"))
                self.supervisor = False
            elif row.event == "lineup":
                if self.supervisor == True or row.type == "regular":
                    self.RegularQueue.append([row.client, row.type])
                    print(row.type,"client", row.client, "lines up at RegularQueue", file=open(output, "a"))
                elif self.supervisor == False and row.type == "VIP":
                    self.VIPStack.append([row.client, row.type])
                    print(row.type, "client", row.client, " lines up at VIPStack", file=open(output, "a"))
            elif row.event == "serve":
                if self.VIPStack != []:
                    print("Now serving", self.VIPStack[-1][0],"from VIPStack", file=open(output, "a"))
                    self.VIPStack.pop()
                else:
                    print("Now serving", self.RegularQueue[0][0],"from RegularQueue", file=open(output, "a"))
                    self.RegularQueue.pop(0)
                    
                

In [75]:
txt = input("Please enter txt file input to be used: ")
df = pd.read_csv(txt, header=None)
df.columns=['event', 'client', 'type']

output = input("Please enter the name of the txt file output to be created: ")
open(output,"w+")
b = CorruptQueue(df)
b.read()

Please enter txt file input to be used:  officeinput.txt
Please enter the name of the txt file output to be created:  test.txt


# Problem 3

In [79]:
class CorruptQueue:
    def __init__ (self):
        self.RegularQueue = []
        self.Lambda = 1 #initial value for lambda, mu, and sigma doesn't matter since it will be reset when CQsimulation is called
        self.mu = 1
        self.sigma = 1
        self.t = 0 #simulation time in minutes
        self.arrivals = 0 #Total number of arrivals
        self.departures = 0 #Total number of departures
        self.clients = 0 #current number of clients in the system
        self.arrivalqueue = []
        self.EL = [self.generateArrivalTime(), float('inf')] # Event list
        self.T = 480 #closing time. After time reaches 480 mins, no more arrivals will be generated. This assumes that the government office is open for 8 hours a day.
        self.arrivaltimes = {} #dictionary containing arrival time of client using key of self.arrival
        self.departuretimes = {} #dictionary containg departure time of client using key of self.departure
                    
    def getLambda(self):
        return self.Lambda
    
    def setLambda(self, x):
        self.Lambda = x
        
    def getMu(self):
        return self.mu
    
    def setMu(self, x):
        self.mu = x
        
    def getSigma(self):
        return self.sigma
        
    def setSigma(self, x):
        self.sigma = x
        
    def generateArrivalTime(self): #Random timepsan of the next arrival of sutomer
        ta = 0 #timespan of the next customer
        a = 0
        while a==0: #for every 1 minute, generate the probability of a client arriving. If no client arrives, go to the next minute. If one/multiple clients are generated, record the minute in which they arrive.
            ta +=1
            a = random.poisson(self.getLambda())
        for arrival in range(a): #append the number of arrivals for that one minute in a queue
            self.arrivalqueue.append(1)
        return ta
            
    
    def generateServiceTime(self): #Random time span of service per client
        return random.normal(loc=self.getMu(), scale=self.getSigma())

In [80]:
class CQSimulation(CorruptQueue):
    def __init__ (self, iterations):
        super().__init__()
        self.iterations = iterations #the number of times the simulation should be repeated
        self.meanwaittimes = [] #stores mean wait times of clients for each simulation
        self.meantimesinsystem = [] #stores mean time in system of clients for each simulation
        self.simulate()
        
    def simulate(self):
        for iteration in range(self.iterations):
            super().__init__() #reset variables
            self.setLambda(float(input("Please input lambda value for simulation " + str(iteration+1) + ": ")))
            self.setMu(float(input("Please input mu value for simulation " + str(iteration+1) + ": ")))
            self.setSigma(float(input("Please input sigma value for simulation " + str(iteration+1) + ": ")))
            
            while True:
                if self.EL[0] <= self.EL[1] and self.EL[0] <= self.T:
                    self.t = self.EL[0]
                    self.arrivals += 1
                    self.clients += 1
                    self.arrivalqueue.pop()
                    if self.arrivalqueue ==[]:    
                        self.EL[0] += self.generateArrivalTime() #generate the next arrival time of clients
                    if self.clients == 1: #if client is the only one in queue, then proceed to service and generate time for service to complete. 
                        self.EL[1] = self.t + self.generateServiceTime() 
                    self.arrivaltimes.update({self.arrivals:self.t}) #collect arrival time data
                elif self.EL[1] <= self.EL[0] and self.EL[1] <= self.T:
                    self.t = self.EL[1]
                    self.departures += 1
                    self.clients -= 1
                    if self.clients == 0: #if queue is empty, then service time becomes infinite
                        self.EL[1] = float('inf')
                    else: #if there are still clients in queue, then generate the service time for the next client.
                        self.EL[1] += self.generateServiceTime()
                    self.departuretimes.update({self.departures:self.t}) #collect departure time data
                elif min(self.EL) > self.T and self.clients > 0:
                    self.t = self.EL[1]
                    self.departures += 1
                    self.clients -= 1
                    if self.clients > 0:
                        self.EL[1] += self.generateServiceTime()
                    self.departuretimes.update({self.departures:self.t}) #collect departure time data
                else:
                    timesinsystem = [] # array containing the time spent in the system by each customer
                    waittimes = [] # array containing the queue wait time of each customer
                    for i in range(self.arrivals):
                        timesinsystem.append(self.departuretimes.get(i+1)-self.arrivaltimes.get(i+1))
                        if  i == 0 or (self.departuretimes.get(i)-self.arrivaltimes.get(i+1)) <= 0:
                            waittimes.append(0)
                        else:
                            waittimes.append(self.departuretimes.get(i)-self.arrivaltimes.get(i+1))
                    self.meanwaittimes.append(np.mean(waittimes))
                    self.meantimesinsystem.append(np.mean(timesinsystem))
                    print("\nSimulation", iteration+1, "Results:\nAverage wait time: ", np.mean(waittimes), "\nAverage time a customer is in the system: ", np.mean(timesinsystem), "\n")
                    break
        print("AVERAGE FOR ALL SIMULATIONS\nAverage wait time of all simulations: ", np.mean(self.meanwaittimes))
        print("Average time a customer is in the system for all simulations:", np.mean(self.meantimesinsystem))

In [81]:
b = CQSimulation(3)

Please input lambda value for simulation 1:  0.5
Please input mu value for simulation 1:  1
Please input sigma value for simulation 1:  0.2



Simulation 1 Results:
Average wait time:  0.37375744584378867 
Average time a customer is in the system:  1.3697194561684833 



Please input lambda value for simulation 2:  0.5
Please input mu value for simulation 2:  1
Please input sigma value for simulation 2:  0.2



Simulation 2 Results:
Average wait time:  0.5388534862134455 
Average time a customer is in the system:  1.5409546869762722 



Please input lambda value for simulation 3:  0.5
Please input mu value for simulation 3:  1
Please input sigma value for simulation 3:  0.2



Simulation 3 Results:
Average wait time:  0.5603098847483697 
Average time a customer is in the system:  1.5495477638719914 

AVERAGE FOR ALL SIMULATIONS
Average wait time of all simulations:  0.4909736056018679
Average time a customer is in the system for all simulations: 1.486740635672249
